In [65]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools
from IPython.display import display

tqdm.pandas()

/Users/kien1/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [66]:
# define column names
col_names = {
    "data": [ 'user id' , 'item id' , 'rating' , 'timestamp'],
    "item": ['movie id' , 'movie title' , 'release date' , 'video release date' ,
              'IMDb URL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
              "Children's" , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western'],
    "user": ['user id' , 'age' , 'gender' , 'occupation' , 'zip code'],
    "genre": ['genre', 'genre id']
}

In [67]:
def read_data(file_name, sep, encoding, col_names):
    output = pd.read_csv(file_name, sep=sep, encoding=encoding, names=col_names)
    return output

In [68]:
ratings = read_data("./u.data", "\t", 'utf-8', col_names["data"])
ratings.head(5)

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [69]:
movies = read_data("./u.item", "|", 'latin-1', col_names["item"])
movies.drop(columns= ['video release date', 'IMDb URL'], inplace=True)
# only for debug
movies = movies[0:100]
movies.head(5)

,movie id,movie title,release date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [70]:
users = read_data("./u.user", "|", 'utf-8', col_names["user"])
users.head(5)

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [71]:
genres = read_data("./u.genre", "|", 'utf-8', col_names["genre"])
genres.head(20)

,genre,genre id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4
5,Comedy,5
6,Crime,6
7,Documentary,7
8,Drama,8
9,Fantasy,9


In [72]:
def get_movies_groupby_genre():
    genres = {k: [] for k in range(19)}
    for index, row in tqdm(movies.iterrows()):
        row_genre = np.array(row[3:])
        genres_index = np.argwhere(row_genre==1)
        for i in genres_index:
            genres[i[0]].append(row["movie title"])
    return genres 

In [73]:
def get_ratings_single_movie(movie_id):
    # get all available ratings for a single movie
    return ratings[ratings["item id"] == movie_id].sort_values(by=['user id'])

In [74]:
def get_ratings_single_user(user_id):
    # get all movies rated by the user
    return ratings[ratings["user id"] == user_id].sort_values(by=['item id'])  

In [75]:
def get_both_rated_set(user1_ratings, user2_ratings):
    # return all items rated by both user sorted by the item id
    sim_user1_ratings = user1_ratings[user1_ratings["item id"].isin(user2_ratings["item id"])]
    sim_user2_ratings = user2_ratings[user2_ratings["item id"].isin(user1_ratings["item id"])]
    
    sim_user1_ratings = sim_user1_ratings.sort_values(by=['item id'])
    sim_user2_ratings = sim_user2_ratings.sort_values(by=['item id'])
    return sim_user1_ratings, sim_user2_ratings

In [76]:
def get_both_raters_set(item1_ratings, item2_ratings):
    # return all ratings bytserh users who rated both items sorted by the user id
    item1_raters = item1_ratings[item1_ratings["user id"].isin(item2_ratings["user id"])]["user id"].tolist()
    item2_raters = item2_ratings[item2_ratings["user id"].isin(item1_ratings["user id"])]["user id"].tolist()
    users_rated_both = np.unique(item1_raters + item2_raters)
    ratings_both = ratings[ratings["user id"].isin(users_rated_both)]
    ratings_both = ratings_both.sort_values(by=['user id'])
    return ratings_both

In [77]:
def get_both_rated_set_for_group(group_ratings, user_ratings):
    # return all items rated by both user sorted by the item id
    sim_group_ratings = group_ratings[group_ratings["movie id"].isin(user_ratings["item id"])]
    sim_user_ratings = user_ratings[user_ratings["item id"].isin(group_ratings["movie id"])]
    
    sim_group_ratings = sim_group_ratings.sort_values(by=['movie id'])
    sim_user_ratings = sim_user_ratings.sort_values(by=['item id'])
    return sim_group_ratings, sim_user_ratings

# Pearson correlation function

In [78]:
def pearson_correlation(user1_id, user2_id):
    # calculate pearson correlation between 2 users
    # step1: get all ratings by 2 users
    user1_ratings = get_ratings_single_user(user1_id)
    user2_ratings = get_ratings_single_user(user2_id)
    
    # step2: get the items rated by both users
    sim_user1_ratings, sim_user2_ratings = get_both_rated_set(user1_ratings, user2_ratings)
    
    # step3: calculate mean ratings ra, rb
    mean_user1_ratings = np.mean(user1_ratings['rating'])
    mean_user2_ratings = np.mean(user2_ratings['rating'])

    # step4: calculate the variance  
    var_1 = np.array(np.subtract(sim_user1_ratings["rating"], [mean_user1_ratings]))
    var_2 = np.array(np.subtract(sim_user2_ratings["rating"], [mean_user2_ratings]))

    # step5: compute the pearson correlation
    numerator = np.sum(var_1*var_2)
    denominator = np.sqrt(np.sum(np.power(var_1, 2)))*np.sqrt(np.sum(np.power(var_2, 2)))

    if denominator == 0:
        # in the case that denominator = 0 return NaN
        return float('NaN')
    else:
        correlation = numerator / denominator
        return correlation


In [79]:
def pearson_correlation_for_group(group_ratings, user_ratings):
    # step2: get the items commonly rated
    sim_group_ratings, sim_user_ratings = get_both_rated_set_for_group(group_ratings, user_ratings)
    
    # step3: calculate mean ratings ra, rb
    mean_group_ratings = np.mean(group_ratings['average'])
    mean_user_ratings = np.mean(user_ratings['rating'])

    # step4: calculate the variance  
    var_1 = np.array(np.subtract(sim_group_ratings["average"], [mean_group_ratings]))
    var_2 = np.array(np.subtract(sim_user_ratings["rating"], [mean_user_ratings]))

    # step5: compute the pearson correlation
    numerator = np.sum(var_1*var_2)
    denominator = np.sqrt(np.sum(np.power(var_1, 2)))*np.sqrt(np.sum(np.power(var_2, 2)))

    if denominator == 0:
        # in the case that denominator = 0 return NaN
        return float('NaN')
    else:
        correlation = numerator / denominator
        return correlation

# User-based prediction function

In [80]:
def predict_single_pair_user(user1_id, user2_id, item_id):
    # predict item's score of user 1 based on user 2
    # step1: get all ratings by 2 users
    user1_ratings = get_ratings_single_user(user1_id)
    user2_ratings = get_ratings_single_user(user2_id)
    
    # step2: get the items rated by both users
    sim_user1_ratings, sim_user2_ratings = get_both_rated_set(user1_ratings, user2_ratings)
    
    # if there is no similar rated item, return nan
    if sim_user1_ratings.empty:
        return [float('NaN'), float('NaN')]
    
    # step3: compute the mean rating of user 2
    mean_user2_ratings = np.mean(user2_ratings['rating'])
    
    # step4: get the pearson correlation
    correlation = pearson_correlation(user1_id, user2_id)
    var_2 = float(user2_ratings[user2_ratings["item id"] == item_id]["rating"]) - mean_user2_ratings

    # step5: return the output
    numerator = (correlation*var_2)
    denominator = correlation
    return [numerator, denominator]

In [81]:
def predict_user_item(user_id, item_id):
    # predict item's score for user
    # if user already rated the item, return the rating
    existing_rating = ratings.loc[(ratings['user id'] == user_id) & (ratings['item id'] == item_id)]
    if not existing_rating.empty:
        return item_id, movies.at[item_id - 1, 'movie title'], existing_rating['rating'].values[0]
        
    # step 1: get user ratings
    user_ratings = get_ratings_single_user(user_id)
    # step 2: compute the mean rating
    mean_user_ratings = np.mean(user_ratings['rating'])
    
    # step 3:  get all other users which rated the item
    users_domain = ratings[ratings["item id"] == item_id]
    
    # step 4: predict for each user in the users domain
    correlations = users_domain.apply(lambda row: predict_single_pair_user(user_id, row["user id"], item_id), axis=1, result_type="expand")
    correlations = np.array(correlations)
    
    # filter all nan, which cause by no same rated item between 2 users
    correlations = correlations[~np.isnan(correlations).any(axis=1), :]

    # step 5: calculate the score and return
    pred_score = mean_user_ratings + np.sum(correlations[:,0]) / np.sum(correlations[:,1])
    return item_id, movies.at[item_id - 1, 'movie title'], pred_score

In [82]:
def predict_user_item(user_id, item_id):
    # predict item's score for user
    # if user already rated the item, return the rating
    existing_rating = ratings.loc[(ratings['user id'] == user_id) & (ratings['item id'] == item_id)]
    if not existing_rating.empty:
        return item_id, movies.at[item_id - 1, 'movie title'], existing_rating['rating'].values[0]
        
    # step 1: get user ratings
    user_ratings = get_ratings_single_user(user_id)
    # step 2: compute the mean rating
    mean_user_ratings = np.mean(user_ratings['rating'])
    
    # step 3:  get all other users which rated the item
    users_domain = ratings[ratings["item id"] == item_id]
    
    # step 4: predict for each user in the users domain
    correlations = users_domain.apply(lambda row: predict_single_pair_user(user_id, row["user id"], item_id), axis=1, result_type="expand")
    correlations = np.array(correlations)
    
    # filter all nan, which cause by no same rated item between 2 users
    correlations = correlations[~np.isnan(correlations).any(axis=1), :]

    # step 5: calculate the score and return
    pred_score = mean_user_ratings + np.sum(correlations[:,0]) / np.sum(correlations[:,1])
    return item_id, movies.at[item_id - 1, 'movie title'], pred_score

In [83]:
def get_predicted_ratings(user_id):
    movies_ratings = movies.progress_apply(lambda row: predict_user_item(user_id, row["movie id"]), axis=1, result_type="expand")
    movies_ratings.columns = ["movie id", "movie title", "pred_rating"]

    return movies_ratings

# Group recommender

In [125]:
def average_aggregation(users, users_ratings):
    # create dataframe with all 3 users' predicted ratings
    data = [users_ratings[0]["movie id"], users_ratings[0]["movie title"], users_ratings[0]["pred_rating"], users_ratings[1]["pred_rating"], users_ratings[2]["pred_rating"]]
    headers = ["movie id", "movie title", "user{} rating".format(str(users[0])), "user{} rating".format(str(users[1])), "user{} rating".format(str(users[2]))]
    all_users_ratings = pd.concat(data, axis=1, keys=headers)

#     # remove ratings under threshold
#     all_users_ratings = all_users_ratings[all_users_ratings[f"user{str(users[0])} rating"] >= 2]
#     all_users_ratings = all_users_ratings[all_users_ratings[f"user{str(users[1])} rating"] >= 2]
#     all_users_ratings = all_users_ratings[all_users_ratings[f"user{str(users[2])} rating"] >= 2]

    all_users_ratings['average'] = all_users_ratings.iloc[:, 1:4].mean(axis=1)

    return all_users_ratings

In [126]:
def group_recommender(users_list):
    users_ratings = []
    for user in users_list:
        user_ratings = get_predicted_ratings(user)
        users_ratings.append(user_ratings)
    
    group_ratings = average_aggregation(users_list, users_ratings)
    
    return group_ratings.sort_values(by=['average'], ascending=False)

# Why-Not Algorithms

In [133]:
def atomic_granularity_reason(all_movies, user_ids, movie_name, ratings, group_ratings, peers, closest_peers, min_rated_num=5, movie_num=20):
    reasons = []
    
    if all_movies.loc[all_movies['movie title'] == movie_name].shape[0] == 0:
        reasons.append("The movie is not in the database")
        return reasons

    movie_id = all_movies.loc[all_movies['movie title'] == movie_name]['movie id'].iloc[0]  
    movie_rating = group_ratings.loc[group_ratings['movie id'] == movie_id]['average'].iloc[0]
    same_rated_movies = group_ratings[group_ratings['average'] == movie_rating]

    first_same_rated_movie_id = same_rated_movies.iloc[0]['movie id']
    reindexed_group_ratings = group_ratings.reset_index(drop=True)

    same_rated_movie_rating_index = reindexed_group_ratings.loc[reindexed_group_ratings['movie id'] == first_same_rated_movie_id].index[0]
    if same_rated_movies.shape[0] > 1 and same_rated_movie_rating_index <= movie_num:
        reasons.append("This movie's predicted rating ties with many others who made the list")
    
    movie_rating_index = reindexed_group_ratings.loc[reindexed_group_ratings['movie id'] == movie_id].index[0]
    if movie_num < movie_rating_index <= 2*movie_num:
        reasons.append(f"This movie is ranked at position {movie_rating_index}. It will appear if you ask for more recommendations.")

    for user_id in user_ids:
        user_rating_movie_row = ratings[(ratings['item id'] == movie_id) & (ratings['user id'] == user_id)]
        if user_rating_movie_row.shape[0] > 0:
            user_rating_movie = user_rating_movie_row['rating'].iloc[0]
            reasons.append(f"A member of the group rated this movie low (only {user_rating_movie})")

    peer_ratings = ratings.loc[ratings['user id'].isin(peers['user id'])]
    movie_ratings = peer_ratings.loc[peer_ratings['item id'] == movie_id]
    if movie_ratings.shape[0] == 0:
        reasons.append("This movie is not rated")

    if movie_ratings.shape[0] > 0:
        most_similar_peers_rated = movie_ratings.loc[movie_ratings['user id'].isin(closest_peers["user id"])]
        if most_similar_peers_rated.shape[0] < min_rated_num:
            reasons.append(f"Not enough similar peers rated this movie, only {most_similar_peers_rated.shape[0]} out of {min_rated_num} did")
        if movie_ratings.shape[0] < min_rated_num:
            reasons.append(f"Not enough peers rated this movie, only {movie_ratings.shape[0]} out of {min_rated_num} did")
    if len(reasons) > 0:
        return reasons

    return ["This movie should be in your recommended list"]
    

In [134]:
# WIP: Adapt to support group granularity (e.g. genre)
# TODO: Choose 100 random users to be peers, then calculate similarity from the beginning. 
def group_granularity_reason(genre, all_genres, all_movies, user_ids, movie_names, ratings, group_ratings, peers, closest_peers, min_rated_num=5, movie_num=20):
    genre_reasons = []
    
    if all_genres.loc[all_genres['genre'] == genre].shape[0] == 0:
        genre_reasons.append("The genre is not in the database")
        return genre_reasons    

    # TODO: Combine answers for all individual items in the same category.
    # Return a user-friendly output.
    reasons = []
    for movie_name in movie_names:
        atomic_reasons = atomic_granularity_reason(movies, user_ids, movie_name, ratings, group_ratings, peers, closest_peers, min_rated_num, movie_num)
        reasons = reasons + atomic_reasons
    # TODO: If a lot of movies have the same reason, that's the general reason for the whole genre
    tie_reason = sum(1 if reason.startswith("This movie's predicted rating ties") else 0 for reason in reasons)
    low_rank_reason = sum(1 if reason.startswith("This movie is ranked at position") else 0 for reason in reasons)
    low_member_score_reason = sum(1 if reason.startswith("A member of the group rated this") else 0 for reason in reasons)
    not_rated_reason = sum(1 if reason.startswith("This movie is not rated") else 0 for reason in reasons)
    not_enough_similar_peers_reason = sum(1 if reason.startswith("Not enough similar peers rated this movie") else 0 for reason in reasons)
    not_enough_peers_reason = sum(1 if reason.startswith("Not enough peers rated this movie") else 0 for reason in reasons)
    movie_shoud_in_list = sum(1 if reason.startswith("This movie should be in your recommended list") else 0 for reason in reasons)
    
    main_reason = np.argmax([tie_reason, low_rank_reason, low_member_score_reason, not_rated_reason, not_enough_similar_peers_reason, not_enough_peers_reason, movie_shoud_in_list])
    
    main_reasons = {
        "0": "Too many tie ratings for movies in this genre",
        "1": f"Movies in this genre are in top {movie_num*2}, show more to see the movies",
        "2": "Your group members rate movies in this genre low score.",
        "3": "Movies of this genre aren't rated",
        "4": f"Your similar peers do not like {genre}",
        "5": f"Not enough peers rate {genre}",
        "6": "This genre should be in your recommended list"
    }
    
    return main_reasons[str(main_reason)]

In [135]:
def position_absenteeism_reason(all_movies, movie_name, ratings, group_ratings, peers, closest_peers, desired_position=1, min_bad_rated_similar_peers=25, movie_num=20):
    reasons = []

    if all_movies.loc[all_movies['movie title'] == movie_name].shape[0] == 0:
        reasons.append("The movie is not in the database")
        return reasons

    movie_id = all_movies.loc[all_movies['movie title'] == movie_name]['movie id'].iloc[0]    
    movie_rating = group_ratings.loc[group_ratings['movie id'] == movie_id]['average'].iloc[0]
    same_rated_movies = group_ratings[group_ratings['average'] == movie_rating]

    first_same_rated_movie_id = same_rated_movies.iloc[0]['movie id']
    reindexed_group_ratings = group_ratings.reset_index(drop=True)

    same_rated_movie_rating_index = reindexed_group_ratings.loc[reindexed_group_ratings['movie id'] == first_same_rated_movie_id].index[0]
    if same_rated_movies.shape[0] > 1 and same_rated_movie_rating_index <= movie_num:
        reasons.append("This movie's predicted rating ties with many others")
        return reasons

    desired_position_movie_rating = reindexed_group_ratings['average'].iloc[desired_position-1]

    peer_ratings = ratings.loc[ratings['user id'].isin(peers['user id'])]
    movie_ratings = peer_ratings.loc[peer_ratings['item id'] == movie_id]
    movie_ratings_low_score = movie_ratings[movie_ratings['rating'] <= desired_position_movie_rating]

    most_similar_peers_rated = movie_ratings.loc[movie_ratings['user id'].isin(closest_peers["user id"])]
    most_similar_peers_rated_low_score = most_similar_peers_rated[most_similar_peers_rated['rating'] <= desired_position_movie_rating]

    if most_similar_peers_rated_low_score.shape[0] >= min_bad_rated_similar_peers:
        reasons.append(f"{most_similar_peers_rated_low_score.shape[0]} of {closest_peers.shape[0]} most similar peers give this movie a lower score")
    if movie_ratings_low_score.shape[0] >= min_bad_rated_similar_peers:
        reasons.append(f"{movie_ratings_low_score.shape[0]} of {peers.shape[0]} peers give this movie a lower score")
    if len(reasons) > 0:
        return reasons
    else:
        return [f"{peers.shape[0] - movie_ratings_low_score.shape[0]} of peers really like it, but {movie_ratings_low_score.shape[0]} do not enjoy it as much"]

In [136]:
def atomic_granularity_question(movie_name, user_ids, group_ratings, peers_num=100, closest_peers_num=10, min_rated_num=5, movie_num=20):    
    # calculate pearson correlation for every user
    peers = users.loc[~users['user id'].isin(user_ids)].sample(peers_num)
    peers["similarity"] = peers.apply(lambda row: pearson_correlation_for_group(group_ratings, get_ratings_single_user(row["user id"])), axis=1)
    closest_peers = peers.sort_values(by=['similarity'], ascending=False).head(closest_peers_num)

    reasons = atomic_granularity_reason(movies, user_ids, movie_name, ratings, group_ratings, peers, closest_peers, min_rated_num, movie_num)
    print("***********************************************************")
    for reason in reasons:
        print(reason)

In [143]:
def group_granularity_question(genre, user_ids, group_ratings, peers_num=100, closest_peers_num=10, min_rated_num=5, movie_num=20):    
    # calculate pearson correlation for every user
    peers = users.loc[~users['user id'].isin(user_ids)].sample(peers_num)
    peers["similarity"] = peers.apply(lambda row: pearson_correlation_for_group(group_ratings, get_ratings_single_user(row["user id"])), axis=1)
    closest_peers = peers.sort_values(by=['similarity'], ascending=False).head(closest_peers_num)

    movies_by_genre = get_movies_groupby_genre();    
    genre_id = genres.loc[genres["genre"] == "Adventure"]["genre id"].iloc[0]
    all_movies = movies_by_genre[genre_id]
    
    reason = group_granularity_reason(genre, genres, all_movies, user_ids, all_movies, ratings, group_ratings, peers, closest_peers, min_rated_num, movie_num)
    print("***********************************************************")
    print(reason)

In [142]:
def position_absenteeism_question(movie_name, user_ids, group_ratings, desired_position=1, peers_num=100, closest_peers_num=10, movie_num=20):    
    # calculate pearson correlation for every user
    peers = users.loc[~users['user id'].isin(user_ids)].sample(peers_num)
    peers["similarity"] = peers.apply(lambda row: pearson_correlation_for_group(group_ratings, get_ratings_single_user(row["user id"])), axis=1)
    closest_peers = peers.sort_values(by=['similarity'], ascending=False).head(closest_peers_num)
    min_bad_rated_similar_peers = closest_peers_num/2

    reasons = position_absenteeism_reason(movies, movie_name, ratings, group_ratings, peers, closest_peers, desired_position, min_bad_rated_similar_peers, movie_num)
    print("***********************************************************")
    for reason in reasons:
        print(reason)

## Some numbers for the experiment
 - 100 peers
 - 10 closest peers
 - For granularity: at least 5 peers need to rate the movie to make it count

### Get group ratings

In [139]:
group_ratings = group_recommender([2, 17, 35])
display(group_ratings.head(20))

100%|██████████| 100/100 [01:22<00:00,  1.21it/s]


,movie id,movie title,user2 rating,user17 rating,user35 rating,average
49,50,Star Wars (1977),5.000000,4.060897,3.267540,4.530449
99,100,Fargo (1996),5.000000,4.000000,3.855796,4.500000
11,12,"Usual Suspects, The (1995)",4.542483,4.126286,2.994006,4.334384
63,64,"Shawshank Redemption, The (1994)",4.550331,3.874734,3.221284,4.212532
97,98,"Silence of the Lambs, The (1991)",4.407805,3.745008,3.839236,4.076406
22,23,Taxi Driver (1976),4.292006,3.763017,2.218471,4.027512
44,45,Eat Drink Man Woman (1994),4.264728,3.762569,3.406358,4.013649
0,1,Toy Story (1995),4.000000,4.000000,2.549545,4.000000
59,60,Three Colors: Blue (1993),4.252207,3.735299,2.909408,3.993753
6,7,Twelve Monkeys (1995),3.973975,4.000000,3.903087,3.986987


In [140]:
len(group_ratings)

100

### Atomic granularity
- Why not _Mighty Aphrodite (1995)_?

In [1096]:
atomic_granularity_question("Mighty Aphrodite (1995)", [2, 17, 35], group_ratings)

***********************************************************
This movie is ranked at position 38. It will appear if you ask for more recommendations.


In [144]:
group_granularity_question("Adventure", [2, 17, 35], group_ratings)

100it [00:00, 7721.61it/s]

***********************************************************
Your similar peers do not like Adventure


### Atomic position absenteeism
- Why not rank _Toy Story (1995)_ second?

In [1097]:
position_absenteeism_question("Toy Story (1995)", [2, 17, 35], group_ratings, 3)

***********************************************************
5 of 10 most similar peers give this movie a lower score
47 of 100 peers give this movie a lower score
